### 4. Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика),  в зависимости от аэропорта вылета 

In [1]:
%time
import pandas as pd
import datetime
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.linear_model import LinearRegression

Wall time: 0 ns


In [2]:
airports = pd.read_csv('airports.csv')
airlines = pd.read_csv('airlines.csv')
flights = pd.read_csv('flights.csv', dtype={'ORIGIN_AIRPORT': str, 'DESTINATION_AIRPORT': str})

### Предобработка

In [4]:
flights = flights.merge(airports[['IATA_CODE']],how='inner', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE').drop('IATA_CODE',axis=1)
flights = flights.merge(airports[['IATA_CODE']],how='inner', left_on='DESTINATION_AIRPORT', right_on='IATA_CODE').drop('IATA_CODE',axis=1)

In [5]:
flights.shape

(5332914, 31)

In [6]:
# удаляем отмененные и перенесенные рейсы
flights = flights[(flights['DIVERTED'] == 0) & (flights['CANCELLED'] == 0)]

In [7]:
flights.shape

(5231130, 31)

In [8]:
%%time
def make_date(df):
  return datetime.date(df.YEAR, df.MONTH, df.DAY)

def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.dDate.dt.year
    data['month'] = data.dDate.dt.month
    data['day'] = data.dDate.dt.day
    data['dayofweek'] = data.dDate.dt.dayofweek
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['ARRIVAL_DELAY'].shift(lag)

flights['dDate'] = flights.apply(make_date,axis = 1)

In [34]:
groups = flights.groupby(['dDate','DESTINATION_AIRPORT'])['ARRIVAL_DELAY'].sum().reset_index()
groups['dDate'] = pd.to_datetime(groups['dDate'])

In [35]:
%%time
departure_to = []
for i in groups[groups.ARRIVAL_DELAY>0].DESTINATION_AIRPORT.unique():
    mod = groups[(groups.DESTINATION_AIRPORT==i)&(groups.ARRIVAL_DELAY>0)][['dDate','ARRIVAL_DELAY']]
    mod.columns = ['dDate','ARRIVAL_DELAY']
    mod = pd.DataFrame(mod)
    
    try:
        make_features(mod,5,2)
        mod.dropna(inplace=True)
        mod.set_index('dDate',inplace=True)
           
        X_train,X_test,y_train, y_test = train_test_split(mod.drop('ARRIVAL_DELAY', axis=1),mod.ARRIVAL_DELAY, shuffle=False, test_size=0.2)
        model_lr = LinearRegression()
        model_lr.fit(X_train,y_train)
        y_predicted_lr = model_lr.predict(X_test)
        
        departure_to.append([i,y_test.mean(),np.sqrt(mean_squared_error(y_test, y_predicted_lr))])

    except Exception as e:
        print('Error', str(e))
        

Wall time: 15 s


In [36]:
departure_to = pd.DataFrame(where_to_go)
departure_to.columns = ['DESTINATION_AIRPORT','MEAN_ARRIVAL_DELAY','RMSE']
departure_to.head()

,DESTINATION_AIRPORT,MEAN_ARRIVAL_DELAY,RMSE
0,ABQ,468.393443,464.546786
1,ACT,91.133333,117.312687
2,ACV,71.687500,72.302740
3,ACY,118.428571,260.628808
4,AEX,139.060000,122.493103


In [51]:
start_airport = np.random.choice(flights['ORIGIN_AIRPORT'].unique()) 
departure_from_start_airport = flights[flights.ORIGIN_AIRPORT==start_airport]['DESTINATION_AIRPORT'].unique()

departure_from_start_airport = pd.DataFrame(departure_from_start_airport)
departure_from_start_airport.columns = ['DESTINATION_AIRPORT']

top3 = departure_from_start_airport\
    .merge(departure_to,on='DESTINATION_AIRPORT',how='inner')\
    .sort_values(by=['RMSE','MEAN_ARRIVAL_DELAY'],ascending=[True, True])\
    .head(3)

print('Топ-3 аэропорта прилета для аэропорта вылета',start_airport,':')
print('')
print(top3)

Топ-3 аэропорта прилета для аэропорта вылета JFK :

   DESTINATION_AIRPORT  MEAN_ARRIVAL_DELAY       RMSE
59                 PSE           48.978261  62.084367
63                 HYA           11.833333  76.250344
60                 BQN           69.543860  90.393756
